# File to convert the Raw file to Taumata Arowai Format

In [28]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

## Reading the raw file 

In [29]:
# Read the input file 
df_inputexcelfile = pd.read_excel('Christchurch Drinking Water 2008 to 2021.xlsx')
# Create a new workbook for the Christchurch file
wb = Workbook()
ws = wb.active

## Removing teh Prefix and Suffix in the value column

In [30]:
# Function to extract the prefix from the 'Result' column
def get_value_prefix(result):
    """Extracts the prefix from the 'Result' column."""
    prefix = ''
    value = ''

    if isinstance(result, str):
        if result.startswith('>=') or result.startswith('<='):
            prefix = result[0:2]
            value = result[2:]
        elif result.startswith('>') or result.startswith('<'):
            prefix = result[0:1]
            value = result[1:]
        else:
            value = result

    return prefix, value


## Reading standardised Determiand List and Unit of Measurements

In [31]:
detDflist = pd.read_csv("C:\water analysis proj\detName_list.csv")
UnitDflist = pd.read_csv(r"C:\water analysis proj\unit_char2Fix_list.csv")

In [33]:
def rename_determ(od):
    odet = od.upper().strip()
    hit = ""

    for index, row in detDflist.iterrows():
        r = list(filter(lambda x: pd.notna(x), row[1:]))  # Filter out missing values
        r = [x.upper() for x in r]

        if odet in r:
            hit = row.iloc[0]
            break
        else:
            hit = od.strip()

    return hit

In [34]:
def rename_units(ounit):
    ounit = str(ounit).upper().strip()
    hit_unit = ""

    for index, row in UnitDflist.iterrows():
        r_unit = list(filter(lambda x: pd.notna(x), row[1:]))  # Filter out missing values
        r_unit = [str(x).upper() for x in r_unit]

        if ounit in r_unit:
            hit_unit = str(row.iloc[0])
            break
        else:
            hit_unit = ounit.strip()

    return hit_unit

## Converting time to a standard format

In [35]:
def convert_time_format(time_str):
    time_str= str(time_str).strip()
    if pd.notna(time_str):
        if ':' in time_str:
            return time_str  # Time is already in the correct format
        elif '.' in time_str:
            return time_str.replace('.', ':')
        elif len(time_str) == 3:
            return f"{time_str[0]}:{time_str[1:]}"
        elif len(time_str) == 4:
            if time_str[0] == '0':
                return f"{time_str[1:3]}:{time_str[3:]}"
            else:
                return f"{time_str[:2]}:{time_str[2:]}"
        else:
            return None  # Invalid time format
    else:
        return None

## Processing the Raw file to Taumata Arowai Format

In [36]:


# Assuming df_abc is your DataFrame
christchurch_data = []

for index, row in df_inputexcelfile.iterrows():
    sample_time = convert_time_format(row['Sample Time'])
    prefix, value = get_value_prefix(row['Result'])
   
    data = {
        'sample_date':pd.to_datetime(row['Date']).strftime('%Y-%m-%d'),
        'sample_date_time': sample_time,
        'local_authority': 'Christchurch City Council',
        'supply_component_id': '',
        'reported_component_id': row['PressureZone'],
        'old_supply_id': row['LocationID'],
        'supply_component_name': '',
        'location_address': row['Address'],
        'latitude': '',
        'longitude': '',
        'parameter_determinand': rename_determ(row['Determinand']),
        'original_determinand_name': row['Determinand'],
        'original_value': row['Result'],
        'value_prefix': prefix,
        'value': value,
        'unit': rename_units(row['Units']),
        'original_unit': row['Units'],
        'laboratory': row['LabRef'],
        'lab_report_code': row['SampleID'],
        'external_sample_id': '',
        'file_name': 'Christchurch Drinking Water 2008 to 2021',
        'LabID':row['LabID'],
        'Purpose':row['Purpose'],
        'Name':row['Name'],
        'Comments':row['Comments'],
        'Status':row['Status']
        }

    christchurch_data.append(data)

# Create a DataFrame from the 'Christchurch' list
df_christchurch = pd.DataFrame(christchurch_data)

# Save the 'Christchurch' DataFrame to a new Excel file
df_christchurch.to_excel('Christchurch_22022024.xlsx', index=False)
